In [2]:
!pip install transformers sentence-transformers

  Using cached nvidia_cudnn_cu12-9.1.0.70-py3-none-manylinux2014_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusolver_cu12-11.6.1.9-py3-none-manylinux2014_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.3.1.170-py3-none-manylinux2014_x86_64.whl.metadata (1.6 kB)
Using cached nvidia_cudnn_cu12-9.1.0.70-py3-none-manylinux2014_x86_64.whl (664.8 MB)
Using cached nvidia_cusolver_cu12-11.6.1.9-py3-none-manylinux2014_x86_64.whl (127.9 MB)
Using cached nvidia_cusparse_cu12-12.3.1.170-py3-none-manylinux2014_x86_64.whl (207.5 MB)
  Attempting uninstall: nvidia-cudnn-cu12
    Found existing installation: nvidia-cudnn-cu12 9.3.0.75
    Uninstalling nvidia-cudnn-cu12-9.3.0.75:
      Successfully uninstalled nvidia-cudnn-cu12-9.3.0.75
  Attempting uninstall: nvidia-cusolver-cu12
    Found existing installation: nvidia-cusolver-cu12 11.6.3.83
    Uninstalling nvidia-cusolver-cu12-11.6.3.83:
      Successfully uninstalled nvidia-cusolver-cu12-11.6.3.83


In [3]:
import math
import random
import pandas as pd
import numpy as np
import json
import torch
import matplotlib.pyplot as plt
import seaborn as sns

from transformers import AutoModelForCausalLM, AutoTokenizer
from sentence_transformers import SentenceTransformer
from IPython.display import clear_output
from tqdm.notebook import tqdm

In [4]:
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV
from sklearn.model_selection import cross_val_score, StratifiedKFold
from sklearn.model_selection import GridSearchCV

In [5]:
seed = 61
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

В данном файле реализуем бейзлайн на классическом ML, используя логистическую регрессию на эмбеддингах.  
В качестве эбеддера выступает `deepvk/user-base`.

In [6]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
data = pd.read_json('/content/drive/My Drive/DLS/Part2/Step Agent/data_final_for_dls_new.jsonl', lines=True)

In [8]:
data.columns = ['text', 'address', 'name', 'norm_name_ru', 'permalink', 'prices_summarized', 'relevance', 'reviews_summarized', 'relevance_new']
train_data = data[570:]
eval_data = data[:570]
train_data = train_data[train_data["relevance"] != 0.1].reset_index(drop=True)
eval_data = eval_data[eval_data["relevance"] != 0.1].reset_index(drop=True)

In [9]:
train_data.head(2)

,text,address,name,norm_name_ru,permalink,prices_summarized,relevance,reviews_summarized,relevance_new
0,налоговая 5007,"Московская область, Королёв, улица Богомолова, 4",Налоговая служба; Межрайонная ИФНС № 2; ИФНС №...,Налоговая инспекция,1377658436,None,0.0,Организация занимается обслуживанием налогопла...,0.0
1,шугаринг Красноярск,"Красноярск, микрорайон Взлётка, улица Весны, 3",Студия красоты Дарлинг; Darling; Дарлинг,Салон красоты,1085001465,Студия красоты «Дарлинг» предоставляет услуги ...,1.0,Студия красоты «Дарлинг» предоставляет бьюти-у...,1.0


**Немного подготовим данные**  
Для этого создадим пару новых колонок. В колонку `text_delimiter` добавим [РАЗДЕЛИТЕЛЬ] к данным из `text`. Возможно это поможет в далнейшем при конкатенации эмбеддингов более явно выделить границу. В колнке `name_plus` объединим данные из `name и norm_name_ru`.  
На этих 2-х новых колонках будут построены эмбеддинги и сконкатенированны в один вектор, значения которого и станут фичами для регрессии.  
Если сделать конкатенацию текстов еще и других колонок (prices_summarized, reviews_summarized) то информации станет слишком много и, вероятно, произойдет размытие "ядровой информации" и (проверено на практике) это ухудшит на 2-3% accuracy.

In [10]:
train_data['text_delimiter'] = (train_data['text'] + ' /// [РАЗДЕЛИТЕЛЬ] ///')
train_data['name_plus'] = (train_data['name'] + ' ' + train_data['norm_name_ru'])

In [11]:
eval_data['text_delimiter'] = (eval_data['text'] + ' /// [РАЗДЕЛИТЕЛЬ] ///')
eval_data['name_plus'] = (eval_data['name'] + ' ' + eval_data['norm_name_ru'])

In [12]:
train_data.head(1)

,text,address,name,norm_name_ru,permalink,prices_summarized,relevance,reviews_summarized,relevance_new,text_delimiter,name_plus
0,налоговая 5007,"Московская область, Королёв, улица Богомолова, 4",Налоговая служба; Межрайонная ИФНС № 2; ИФНС №...,Налоговая инспекция,1377658436,None,0.0,Организация занимается обслуживанием налогопла...,0.0,налоговая 5007 /// [РАЗДЕЛИТЕЛЬ] ///,Налоговая служба; Межрайонная ИФНС № 2; ИФНС №...


**Получаем эмбеддинги**  
сначала трейн, затем тест

In [13]:
# Загружаем модель
model = SentenceTransformer('deepvk/user-base')
# Получаем эмбеддинги для текстов
emb_text_delimiter = model.encode(train_data['text_delimiter'].astype(str).tolist(), show_progress_bar=True, device='cuda')
emb_name_plus = model.encode(train_data['name_plus'].astype(str).tolist(), show_progress_bar=True, device='cuda')

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/338 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/232 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/964 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

Batches:   0%|          | 0/935 [00:00<?, ?it/s]

Batches:   0%|          | 0/935 [00:00<?, ?it/s]

In [14]:
# Преобразуем эмбеддинги в numpy-массив (размерность [n_samples, 784])
emb_text_delimiter = np.array(emb_text_delimiter)
emb_name_plus = np.array(emb_name_plus)

# Создаем колонки с эмбеддингами (имена фичей - числа)
columns_delimiter = [str(i) + '_del' for i in range(emb_text_delimiter.shape[1])]  # ['0', '1', ..., 'N']
columns_name_plus = [str(i) + '_plus' for i in range(emb_name_plus.shape[1])]

# Создаем DataFrame
df_delimiter = pd.DataFrame(emb_text_delimiter, columns=columns_delimiter)
df_name_plus = pd.DataFrame(emb_name_plus, columns=columns_name_plus)

In [15]:
train_data.shape, df_delimiter.shape, df_name_plus.shape

((29891, 11), (29891, 768), (29891, 768))

In [16]:
df_delimiter.to_csv('df_delimiter_deepvk_train.csv', index=False, sep=';')
df_name_plus.to_csv('df_name_plus_deepvk_train.csv', index=False, sep=';')

In [ ]:
# df = pd.read_csv('df_delimiter_deepvk.csv', encoding='windows-1251', sep=';')
# df = pd.read_csv('df_name_plus_deepvk.csv', encoding='windows-1251', sep=';')

In [ ]:
train_data.shape, df_delimiter.shape, df_name_plus.shape

((29891, 8), (29891, 768), (29891, 768))

In [17]:
df_train_embed = pd.concat([df_delimiter, df_name_plus], axis=1)

**тест эмбеды**

In [18]:
# Загружаем модель
model = SentenceTransformer('deepvk/user-base')
# Получаем эмбеддинги для текстов
emb_text_delimiter_tst = model.encode(eval_data['text_delimiter'].astype(str).tolist(), show_progress_bar=True, device='cuda')
emb_name_plus_tst = model.encode(eval_data['name_plus'].astype(str).tolist(), show_progress_bar=True, device='cuda')

Batches:   0%|          | 0/16 [00:00<?, ?it/s]

Batches:   0%|          | 0/16 [00:00<?, ?it/s]

In [19]:
# Преобразуем эмбеддинги в numpy-массив (размерность [n_samples, 784])
emb_text_delimiter_tst = np.array(emb_text_delimiter_tst)
emb_name_plus_tst = np.array(emb_name_plus_tst)

# Создаем колонки с эмбеддингами (имена фичей - числа)
columns_delimiter_tst = [str(i) + '_del' for i in range(emb_text_delimiter_tst.shape[1])]  # ['0', '1', ..., 'N']
columns_name_plus_tst = [str(i) + '_plus' for i in range(emb_name_plus_tst.shape[1])]

# Создаем DataFrame
df_delimiter_tst = pd.DataFrame(emb_text_delimiter_tst, columns=columns_delimiter_tst)
df_name_plus_tst = pd.DataFrame(emb_name_plus_tst, columns=columns_name_plus_tst)

In [20]:
df_delimiter_tst.to_csv('df_delimiter_deepvk_tst.csv', index=False, sep=';')
df_name_plus_tst.to_csv('df_name_plus_deepvk_tst.csv', index=False, sep=';')

In [21]:
df_test_embed = pd.concat([df_delimiter_tst, df_name_plus_tst], axis=1)

**Реализуем ЛОГ РЕГ**

In [22]:
X = df_train_embed
y = train_data['relevance']

In [23]:
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.15, random_state=seed, stratify=y)

In [24]:
model = LogisticRegression(solver='liblinear', penalty='l2', C=1, tol=1e-4, multi_class='ovr')
model.fit(X_train, y_train)

/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_logistic.py:1256: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. Use OneVsRestClassifier(LogisticRegression(..)) instead. Leave it to its default value to avoid this warning.
  warnings.warn(


LogisticRegression(C=1, multi_class='ovr', solver='liblinear')

**Смотрим метрику - accuracy**

метрика на трейне и валидации

In [25]:
print(model.score(X_train, y_train))
print(model.score(X_val, y_val))

0.7260203880820246
0.7107493309545049


метрика на тесте - сначала на старой разметке, затем на новой.

In [26]:
X_test = df_test_embed
y_test = eval_data['relevance']
y_test_new = eval_data['relevance_new']

In [27]:
print(model.score(X_test, y_test), model.score(X_test, y_test_new))

0.724 0.672


**ВЫВОДЫ**

Данный алгоритм позволяет получить довольно скромные показатели на уровне 72% на старой разметке и 67% на новой.  
Очевидное ухудшение на новой разметке объясняется тем, что модель обученная на большом объеме зашумленных данных выучивает "ошибочные" закономерности. Затем, очевидно, в тестовой выборке она "узнает" часть этих ошибочных закономерностей, но разметка тут уже иная, по сути из другого распределения.  
В этой связи, пожалуй, целесообразно использовать "запасные" методы получения бейзлайна, такие как разметка через LLM.